In [1]:
%run lib.ipynb

In [2]:
%run make_datapath.ipynb

5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg


In [3]:
%run dataset.ipynb

5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg
5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg
[[ 0.09        0.03003003  0.998       0.996997   18.        ]
 [ 0.122       0.56756757  0.164       0.72672673 14.        ]]
5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg
[[ 0.09        0.03003003  0.998       0.996997   18.        ]
 [ 0.122       0.56756757  0.164       0.72672673 14.        ]]
torch.Size([4, 3, 300, 300])
4
torch.Size([1, 5])


In [4]:
%run transform.ipynb

5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg
5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg
[[ 0.09        0.03003003  0.998       0.996997   18.        ]
 [ 0.122       0.56756757  0.164       0.72672673 14.        ]]


In [5]:
%run extract_infor_annotation.ipynb

5717
.\data\VOCdevkit\VOC2012\JPEGImages\2008_000008.jpg
[[ 0.09        0.03003003  0.998       0.996997   18.        ]
 [ 0.122       0.56756757  0.164       0.72672673 14.        ]]


In [6]:
%run model.ipynb

torch.Size([8732, 4])
SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padd

In [7]:
%run multiboxloss.ipynb
%run utils/augmentation.ipynb

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
torch.backends.cudnn.benchmark = True

device: cpu


In [9]:
# dataloader
root_path = "./data/VOCdevkit/VOC2012"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(root_path)

classes = ["aeroplane", "bicycle", "bird",  "boat", "bottle", 
    "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant",
    "sheep", "sofa", "train", "tvmonitor"]

color_mean = (104, 117, 123)
input_size = 300

#img_list, anno_list, phase, transform, anno_xml
train_dataset = MyDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(input_size, color_mean), anno_xml=Anno_xml(classes))
val_dataset = MyDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(input_size, color_mean), anno_xml=Anno_xml(classes))

batch_size = 32
train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=my_collate_fn)
val_dataloader = data.DataLoader(val_dataset, batch_size, shuffle=False, collate_fn=my_collate_fn)
dataloader_dict = {
    "train": train_dataloader, 
    "val": val_dataloader
}

In [10]:
# network
cfg = {
    "num_classes": 21, #VOC data include 20 class + 1 background class
    "input_size": 300, #SSD300
    "bbox_aspect_num": [4, 6, 6, 6, 4, 4], # Tỷ lệ khung hình cho source1->source6`
    "feature_maps": [38, 19, 10, 5, 3, 1],
    "steps": [8, 16, 32, 64, 100, 300], # Size of default box
    "min_size": [30, 60, 111, 162, 213, 264], # Size of default box
    "max_size": [60, 111, 162, 213, 264, 315], # Size of default box
    "aspect_ratios": [[2], [2,3], [2,3], [2,3], [2], [2]]
}

net = SSD(phase="train", cfg=cfg)
vgg_weights = torch.load("./data/weights/vgg16_reducedfc.pth")
net.vgg.load_state_dict(vgg_weights)

<All keys matched successfully>

In [11]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

In [12]:
# He init
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [13]:
# MultiBoxLoss
criterion = MultiBoxLoss(jaccard_threshold=0.5, neg_pos=3, device=device)

# optimizer
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

In [14]:
# training, validation
def train_model(net, dataloader_dict, criterion, optimizer, num_epochs):
    # move network to GPU
    net.to(device)

    iteration = 1
    epoch_train_loss = 0.0
    epoch_val_loss = 0.0
    logs = []
    for epoch in range(num_epochs+1):
        t_epoch_start = time.time()
        t_iter_start = time.time()
        print("---"*20)
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        print("---"*20)
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
                print("(Training)")
            else:
                if (epoch+1) % 10 == 0:
                    net.eval() 
                    print("---"*10)
                    print("(Validation)")
                else:
                    continue
            for images, targets in dataloader_dict[phase]:
                # move to GPU
                images = images.to(device)
                targets = [ann.to(device) for ann in targets]
                # init optimizer
                optimizer.zero_grad()
                # forward
                with torch.set_grad_enabled(phase=="train"):
                    outputs = net(images)
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    if phase == "train":
                        loss.backward() # calculate gradient
                        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
                        optimizer.step() # update parameters

                        if (iteration % 10) == 0:
                            t_iter_end = time.time()
                            duration = t_iter_end - t_iter_start
                            print("Iteration {} || Loss: {:.4f} || 10iter: {:.4f} sec".format(iteration, loss.item(), duration))
                            t_iter_start = time.time()
                        epoch_train_loss += loss.item()
                        iteration += 1
                    else:
                        epoch_val_loss += loss.item()
        t_epoch_end = time.time()
        print("---"*20)
        print("Epoch {} || epoch_train_loss: {:.4f} || Epoch_val_loss: {:.4f}".format(epoch+1, epoch_train_loss, epoch_val_loss))           
        print("Duration: {:.4f} sec".format(t_epoch_end - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {"epoch": epoch+1, "train_loss": epoch_train_loss, "val_loss": epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("./data/ssd_logs.csv")
        epoch_train_loss = 0.0
        epoch_val_loss = 0.0
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), "./data/weights/ssd300_" + str(epoch+1) + ".pth")

In [15]:
num_epochs = 30
train_model(net, dataloader_dict, criterion, optimizer, num_epochs=num_epochs)

------------------------------------------------------------
Epoch 1/30
------------------------------------------------------------
(Training)


C:\Users\HP\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Iteration 10 || Loss: 13.9786 || 10iter: 590.7018 sec
Iteration 20 || Loss: 11.8258 || 10iter: 470.5018 sec
Iteration 30 || Loss: 11.9142 || 10iter: 454.1413 sec
Iteration 40 || Loss: 10.0509 || 10iter: 485.2065 sec
Iteration 50 || Loss: 8.4861 || 10iter: 501.8508 sec
Iteration 60 || Loss: 7.7305 || 10iter: 5503.9769 sec
Iteration 70 || Loss: 8.0202 || 10iter: 453.5588 sec
Iteration 80 || Loss: 7.5931 || 10iter: 3277.7980 sec
Iteration 90 || Loss: 7.9180 || 10iter: 610.3388 sec
Iteration 100 || Loss: 7.9220 || 10iter: 600.7255 sec
Iteration 110 || Loss: 7.4615 || 10iter: 527.5931 sec
Iteration 120 || Loss: 7.4421 || 10iter: 570.6170 sec
Iteration 130 || Loss: 7.5919 || 10iter: 589.7096 sec
Iteration 140 || Loss: 7.6607 || 10iter: 546.1700 sec
Iteration 150 || Loss: 7.0752 || 10iter: 529.7249 sec
Iteration 160 || Loss: 8.2080 || 10iter: 448.2675 sec
Iteration 170 || Loss: 8.8018 || 10iter: 448.8072 sec
------------------------------------------------------------
Epoch 1 || epoch_train_l

Iteration 1200 || Loss: 5.6359 || 10iter: 499.7130 sec
Iteration 1210 || Loss: 5.8819 || 10iter: 488.9944 sec
Iteration 1220 || Loss: 5.6347 || 10iter: 463.2631 sec
Iteration 1230 || Loss: 5.8228 || 10iter: 457.1816 sec
Iteration 1240 || Loss: 5.8953 || 10iter: 528.6156 sec
Iteration 1250 || Loss: 5.6431 || 10iter: 541.7407 sec
------------------------------------------------------------
Epoch 7 || epoch_train_loss: 1010.0641 || Epoch_val_loss: 0.0000
Duration: 9363.8852 sec
------------------------------------------------------------
Epoch 8/30
------------------------------------------------------------
(Training)
Iteration 1260 || Loss: 5.3497 || 10iter: 334.1845 sec
Iteration 1270 || Loss: 6.1366 || 10iter: 469.7867 sec
Iteration 1280 || Loss: 5.4393 || 10iter: 469.5996 sec
Iteration 1290 || Loss: 5.2509 || 10iter: 514.0397 sec
Iteration 1300 || Loss: 5.5637 || 10iter: 592.5029 sec
Iteration 1310 || Loss: 5.4661 || 10iter: 585.0431 sec
Iteration 1320 || Loss: 5.4588 || 10iter: 601.

Iteration 2330 || Loss: 5.0909 || 10iter: 132.8919 sec
Iteration 2340 || Loss: 4.5417 || 10iter: 448.0674 sec
Iteration 2350 || Loss: 4.9058 || 10iter: 446.2590 sec
Iteration 2360 || Loss: 4.9820 || 10iter: 446.3496 sec
Iteration 2370 || Loss: 4.6940 || 10iter: 445.5308 sec
Iteration 2380 || Loss: 4.9069 || 10iter: 444.6852 sec
Iteration 2390 || Loss: 4.9435 || 10iter: 446.8848 sec
Iteration 2400 || Loss: 4.6683 || 10iter: 445.5661 sec
Iteration 2410 || Loss: 4.6427 || 10iter: 445.9157 sec
Iteration 2420 || Loss: 5.1169 || 10iter: 446.2634 sec
Iteration 2430 || Loss: 4.8300 || 10iter: 446.3721 sec
Iteration 2440 || Loss: 4.9427 || 10iter: 447.6755 sec
Iteration 2450 || Loss: 4.2735 || 10iter: 446.2807 sec
Iteration 2460 || Loss: 4.5549 || 10iter: 446.0612 sec
Iteration 2470 || Loss: 4.8694 || 10iter: 445.4319 sec
Iteration 2480 || Loss: 4.6864 || 10iter: 445.5392 sec
Iteration 2490 || Loss: 4.1771 || 10iter: 446.1700 sec
Iteration 2500 || Loss: 4.7147 || 10iter: 455.6794 sec
----------

Iteration 3500 || Loss: 4.2407 || 10iter: 440.2344 sec
Iteration 3510 || Loss: 4.7038 || 10iter: 438.5743 sec
Iteration 3520 || Loss: 4.1077 || 10iter: 439.5806 sec
Iteration 3530 || Loss: 4.0186 || 10iter: 439.3605 sec
Iteration 3540 || Loss: 4.6108 || 10iter: 439.3688 sec
Iteration 3550 || Loss: 4.5390 || 10iter: 438.7417 sec
Iteration 3560 || Loss: 3.7025 || 10iter: 441.2500 sec
Iteration 3570 || Loss: 4.4771 || 10iter: 439.6412 sec
Iteration 3580 || Loss: 4.4708 || 10iter: 424.4191 sec
------------------------------
(Validation)
------------------------------------------------------------
Epoch 20 || epoch_train_loss: 778.1371 || Epoch_val_loss: 793.0571
Duration: 10764.3231 sec
------------------------------------------------------------
Epoch 21/30
------------------------------------------------------------
(Training)
Iteration 3590 || Loss: 4.5603 || 10iter: 438.8048 sec
Iteration 3600 || Loss: 4.1020 || 10iter: 439.3543 sec
Iteration 3610 || Loss: 4.4116 || 10iter: 437.5573 se

------------------------------------------------------------
Epoch 26 || epoch_train_loss: 717.6158 || Epoch_val_loss: 0.0000
Duration: 8343.7173 sec
------------------------------------------------------------
Epoch 27/30
------------------------------------------------------------
(Training)
Iteration 4660 || Loss: 4.0326 || 10iter: 288.5624 sec
Iteration 4670 || Loss: 4.0917 || 10iter: 442.8758 sec
Iteration 4680 || Loss: 3.6855 || 10iter: 440.6144 sec
Iteration 4690 || Loss: 3.9736 || 10iter: 440.6817 sec
Iteration 4700 || Loss: 3.8010 || 10iter: 442.0962 sec
Iteration 4710 || Loss: 3.7490 || 10iter: 440.3826 sec
Iteration 4720 || Loss: 3.5014 || 10iter: 439.6896 sec
Iteration 4730 || Loss: 3.8625 || 10iter: 448.2385 sec
Iteration 4740 || Loss: 3.9837 || 10iter: 458.4973 sec
Iteration 4750 || Loss: 4.0366 || 10iter: 450.0784 sec
Iteration 4760 || Loss: 4.0354 || 10iter: 452.0952 sec
Iteration 4770 || Loss: 4.3064 || 10iter: 448.8162 sec
Iteration 4780 || Loss: 3.7704 || 10iter: 443